# Parametri

In [ ]:
IMAGE_SIZE = 64
BATCH_SIZE = 16
EPOCHS = 3
# %load_ext tensorboard

# Knjižnice

In [ ]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

# Uvoz učnih podatkov

In [ ]:
zip_path = 'podatki2.zip'
extract_path = 'podatki'

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

data_dir = extract_path

# Razdelitev učnih podatkov

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2, fill_mode='nearest').flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2).flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)))

# Priprava modela

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False

model = Sequential([
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Treniranje modela

In [ ]:
#tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

In [ ]:
model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator, class_weight=class_weights, callbacks=[early_stopping, reduce_lr])

In [ ]:
#%tensorboard --logdir logs/train

In [ ]:
model.save('2FA_model_v2.keras')

# Predikcija

In [ ]:
def prediction(path, model, class_mapping=None):
    image = img_to_array(load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    prob = prediction[0][0]
    predicted_class = 1 if prob > 0.5 else 0
    class_name = ""
    if class_mapping:
        inv_mapping = {v: k for k, v in class_mapping.items()}
        class_name = f" ({inv_mapping.get(predicted_class, 'unknown')})"
    print(f"Raw prediction value: {prob:.4f}")
    print(f"Predicted Class: {predicted_class}{class_name}")
    print(f"Confidence: {prob if predicted_class == 1 else 1-prob:.4f}")

# Predikcija

In [ ]:
prediction("jozi.jpeg", model)


# Nalaganje modela

In [ ]:
model = load_model("2FA_model_v2.keras")

# Croppanje obraza

In [ ]:
import cv2
import numpy as np
import os

def crop_face(image_path):
    input_dir = os.path.dirname(image_path)
    filename = os.path.basename(image_path)
    name, ext = os.path.splitext(filename)

    if not input_dir:
        input_dir = "."

    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image from {image_path}")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        print("No faces detected in the image.")
        return

    if len(faces) > 0:
        x, y, w, h = faces[0]
        face_crop = image[y:y+h, x:x+w]
        output_filename = f"{name}_output{ext}"
        output_path = os.path.join(input_dir, output_filename)
        cv2.imwrite(output_path, face_crop)
        print(f"Saved: {output_path}")

image_path = "jan.jpg"
crop_face(image_path)

In [ ]:
if __name__ == "__main__":
    image_path = "jan.jpg"
    crop_faces(image_path)